# carbon_casefile

In [1]:
def carbon_casefile(net, fuel_dict):
    """
    Assigns GCI values to generators in the pandapower network based on fuel type and emission type.
    """
    # Lookup tables for CO2 and CO2e emissions
    fuel_lookup = {  # CO2 GCI
        "ANT": 0.9095,  # Anthracite Coal
        "BIT": 0.8204,  # Bituminous Coal
        "Oil": 0.7001,  # Heavy Oil
        "GAS": 0.5173,
        "CCGT": 0.3621,  # Gas Combined Cycle
        "ICE": 0.6030,  # Internal Combustion Engine
        "Thermal": 0.6874,  # Thermal Power (General)
        "NUC": 0.0,  # Nuclear Power
        "RE": 0.0,  # Renewable Energy
        "HYD": 0.0,  # Hydropower
        "N/A": 0.0  # Default case
    }

    fuel_lookup_co2e = {  # CO2 GCI + (CH4 GCI * 21) + (N2O GCI * 310)
        "ANT": 0.9143,  # Anthracite Coal
        "BIT": 0.8230,  # Bituminous Coal
        "Oil": 0.7018,  # Heavy Oil
        "GAS": 0.5177,
        "CCGT": 0.3625,  # Gas Combined Cycle
        "ICE": 0.6049,  # Internal Combustion Engine
        "Thermal": 0.6894,  # Thermal Power (General)
        "NUC": 0.0,  # Nuclear Power
        "RE": 0.0,  # Renewable Energy
        "HYD": 0.0,  # Hydropower
        "N/A": 0.0  # Default case
    }

    # Iterate over fuel_dict, assigning GCI values to the corresponding generators in pandapower
    for bus_id, info in fuel_dict.items():
        if "type" in info and "emissions" in info:
            lookup_table = fuel_lookup_co2e if info["emissions"] == "CO2e" else fuel_lookup
            
            # Find matching generators based on `bus` column in `net.gen`
            gen_mask = net.gen["bus"] == bus_id
            net.gen.loc[gen_mask, "GCI"] = lookup_table.get(info["type"], 0.0)  # Assign GCI value safely

In [2]:
def fuel_dict_generation(net):
    """
    Initializes a fuel dictionary from the pandapower network, setting default fuel type and emissions for each generator bus.
    """
    fuel_dict = {}

    for bus_id in net.gen["bus"].unique():  # Extract unique generator bus IDs
        fuel_dict[bus_id] = {"type": "N/A", "emissions": "CO2"}  # Default values

    return fuel_dict

# Example

In [3]:
# Import required libraries
import pandapower as pp
import pandapower.networks as pn
import pandas as pd

In [4]:
# Load the pandapower network (assumed to be in MATPOWER format)
net = pn.case30() 

In [5]:
# Generate the fuel dictionary
fuel_dict = fuel_dict_generation(net)

In [6]:
fuel_dict

{1: {'type': 'N/A', 'emissions': 'N/A'},
 21: {'type': 'N/A', 'emissions': 'N/A'},
 26: {'type': 'N/A', 'emissions': 'N/A'},
 22: {'type': 'N/A', 'emissions': 'N/A'},
 12: {'type': 'N/A', 'emissions': 'N/A'}}

In [7]:
# Assign fuel types and emissions to specific generator buses
fuel_dict[1] = {"type": "GAS", "emissions": "CO2e"}
fuel_dict[12] = {"type": "ANT", "emissions": "CO2"}
fuel_dict[22] = {"type": "BIT", "emissions": "CO2"}

In [8]:
fuel_dict

{1: {'type': 'GAS', 'emissions': 'CO2e'},
 21: {'type': 'N/A', 'emissions': 'N/A'},
 26: {'type': 'N/A', 'emissions': 'N/A'},
 22: {'type': 'BIT', 'emissions': 'CO2'},
 12: {'type': 'ANT', 'emissions': 'CO2'}}

In [9]:
# Apply carbon constraints to the pandapower network
carbon_casefile(net, fuel_dict)

In [10]:
net.gen

,bus,controllable,in_service,name,p_mw,scaling,sn_mva,type,vm_pu,slack,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar,GCI
0,1,True,True,None,60.97,1.0,NaN,None,1.0,False,80.0,0.0,60.0,-20.0,0.5177
1,21,True,True,None,21.59,1.0,NaN,None,1.0,False,50.0,0.0,62.5,-15.0,0.0000
2,26,True,True,None,26.91,1.0,NaN,None,1.0,False,55.0,0.0,48.7,-15.0,0.0000
3,22,True,True,None,19.20,1.0,NaN,None,1.0,False,30.0,0.0,40.0,-10.0,0.8204
4,12,True,True,None,37.00,1.0,NaN,None,1.0,False,40.0,0.0,44.7,-15.0,0.9095


In [11]:
# Run OPF
pp.runopp(net)

In [12]:
# Calculate carbon emissions for each generator
net.res_gen["carbon emission"]=net.res_gen["p_mw"]*net.gen["GCI"]

In [13]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu,carbon emission
0,61.237460,-19.999710,0.508009,0.986762,31.702633
1,22.924613,11.658752,-0.593612,1.002674,0.000000
2,51.877455,19.732024,4.351380,1.049671,0.000000
3,16.213665,32.709865,-0.770482,1.067925,13.301691
4,39.999991,36.245106,2.702687,1.081294,36.379992
